In [1]:
import pandas as pd
import numpy as np
from statsforecast import StatsForecast
import matplotlib.pyplot as plt
#import seaborn as sns
import datetime as dt
import os
import warnings
import matplotlib.dates as mpl_dates
from statsmodels.tsa.seasonal import seasonal_decompose
import operator
from mlforecast import MLForecast
from mlforecast.lag_transforms import Combine, RollingMean
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestRegressor
from statsforecast.models import AutoARIMA, AutoETS, Naive, RandomWalkWithDrift, SeasonalNaive, SeasonalWindowAverage, WindowAverage
import utilsforecast.losses as ufl
from utilsforecast.evaluation import evaluate



c:\Users\pc\Desktop\github-prj\BF-final\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Loading data
future_values = pd.read_csv('future_values.csv',parse_dates = ['date']).rename(columns={'date':'ds','store_id':'unique_id'})
metadata = pd.read_csv('metadata.csv').rename(columns={'store_id':'unique_id'})
sales_data = pd.read_csv('sales_data.csv',parse_dates = ['date']).rename(columns={'date':'ds','store_id':'unique_id','sales':'y'})
sales_data.dtypes

C:\Users\pc\AppData\Local\Temp\ipykernel_15952\594441786.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  sales_data = pd.read_csv('sales_data.csv',parse_dates = ['date']).rename(columns={'date':'ds','store_id':'unique_id','sales':'y'})


unique_id                 object
ds                datetime64[ns]
y                          int64
customers                  int64
open                       int64
promo                      int64
state_holiday             object
school_holiday             int64
dtype: object

In [3]:
# Grouping data by each time series
grouped = sales_data.groupby('unique_id')
summary = grouped.agg(
    count_observed=('ds', 'count'),
    start_date=('ds', 'min'),
    end_date=('ds', 'max')
).reset_index()
print(summary)

     unique_id  count_observed start_date   end_date
0      store_1             924 2013-01-07 2015-07-19
1     store_10             924 2013-01-07 2015-07-19
2    store_100             924 2013-01-07 2015-07-19
3    store_101             924 2013-01-07 2015-07-19
4    store_102             924 2013-01-07 2015-07-19
..         ...             ...        ...        ...
671   store_95             924 2013-01-07 2015-07-19
672   store_96             924 2013-01-07 2015-07-19
673   store_97             924 2013-01-07 2015-07-19
674   store_98             924 2013-01-07 2015-07-19
675   store_99             924 2013-01-07 2015-07-19

[676 rows x 4 columns]


In [4]:
# Grouping data by each time series
grouped = future_values.groupby('unique_id')
summary = grouped.agg(
    count_observed=('ds', 'count'),
    start_date=('ds', 'min'),
    end_date=('ds', 'max')
).reset_index()
print(summary)

     unique_id  count_observed start_date   end_date
0      store_1              60 2015-07-20 2015-09-17
1     store_10              60 2015-07-20 2015-09-17
2    store_100              60 2015-07-20 2015-09-17
3    store_101              60 2015-07-20 2015-09-17
4    store_102              60 2015-07-20 2015-09-17
..         ...             ...        ...        ...
671   store_95              60 2015-07-20 2015-09-17
672   store_96              60 2015-07-20 2015-09-17
673   store_97              60 2015-07-20 2015-09-17
674   store_98              60 2015-07-20 2015-09-17
675   store_99              60 2015-07-20 2015-09-17

[676 rows x 4 columns]


In [5]:
#Checking na 
future_values.isna ().sum ()
metadata.isna ().sum ()
sales_data.isna().sum()

unique_id         0
ds                0
y                 0
customers         0
open              0
promo             0
state_holiday     0
school_holiday    0
dtype: int64

In [6]:
sales_merged = pd.merge(sales_data, metadata, on='unique_id', how='left')
sales_merged.head()

,unique_id,ds,y,customers,open,promo,state_holiday,school_holiday,store_type,assortment,competition_distance
0,store_1,2015-07-19,0,0,0,0,0,0,c,a,1270.0
1,store_2,2015-07-19,0,0,0,0,0,0,a,a,14130.0
2,store_3,2015-07-19,0,0,0,0,0,0,a,c,24000.0
3,store_4,2015-07-19,0,0,0,0,0,0,a,a,7520.0
4,store_5,2015-07-19,0,0,0,0,0,0,a,c,2030.0


In [10]:
sales_merged['ds'] = pd.to_datetime(sales_merged['ds'])

# 创建一个表示“这一行属于哪一周”的新列，按周一为每周起始（你可以改为 W-SUN）
sales_merged['week'] = sales_merged['ds'].dt.to_period('W-MON').dt.start_time

# 聚合：按 store_id 和 week 聚合，取和
weekly_data = sales_merged.groupby(['unique_id', 'week'], as_index=False).agg({
    'y': 'sum',
    'customers': 'sum',
    'promo': 'sum',
    'open': 'sum',
    'state_holiday': 'sum',      # 或可替换为 'max'，视后续建模而定
    'school_holiday': 'sum'
})

TypeError: can only concatenate str (not "int") to str